In [2]:
import pandas as pd
import requests
import mysql.connector

In [6]:
API_KEY = ''
CHANNEL_ID = 'UC2UXDak6o7rBm23k3Vv5dww'
PAGE_TOKEN = ''

searchUrl = 'https://www.googleapis.com/youtube/v3/search?key=' + API_KEY + '&part=snippet' + '&type=video' + '&channelId=UCG5qGWdu8nIRZqJ_GgDwQ-w' + '&order=date' + '&maxResults=50' + '&pageToken='
videosUrl = 'https://www.googleapis.com/youtube/v3/videos?key=' + API_KEY + '&part=snippet,contentDetails,statistics' + '&id='

df = pd.DataFrame(columns=['videoId', 'publishedAt', 'title', 'tags', 'duration', 'viewCount', 'likeCount', 'commentCount'])

In [7]:
while True:
    try:
        searchResponse = requests.get(searchUrl + PAGE_TOKEN).json()
        PAGE_TOKEN = searchResponse['nextPageToken']
        print(len(df.index))

        for video in searchResponse['items']:
            videoId = video['id']['videoId']
            videoResponse = requests.get(videosUrl + videoId).json()
            
            publishedAt = videoResponse['items'][0]['snippet']['publishedAt']
            title = videoResponse['items'][0]['snippet']['title']
            tags = videoResponse['items'][0]['snippet']['tags']

            duration = videoResponse['items'][0]['contentDetails']['duration']

            viewCount = videoResponse['items'][0]['statistics']['viewCount']
            likeCount = videoResponse['items'][0]['statistics']['likeCount']
            commentCount = videoResponse['items'][0]['statistics']['commentCount']

            df = df.append({'videoId' : videoId, 
                            'publishedAt' : publishedAt, 
                            'title' : title, 
                            'tags' : tags,
                            'duration' : duration,
                            'viewCount' : viewCount,
                            'likeCount' : likeCount,
                            'commentCount' : commentCount}, ignore_index=True)
    except:
        print(len(df.index))
        break

0
50
100
150
200
250
300
350
400
450
500


In [10]:
df

#df.to_csv("videos.csv")

In [90]:
mydb = mysql.connector.connect(
  host='localhost',
  user='root',
  password='Justocker70',
  database ='dfdb'
)
print(mydb)
mycursor = mydb.cursor()

In [41]:
#mycursor.execute('CREATE DATABASE dfdb')

#mycursor.execute('SHOW DATABASES')
#for db in mycursor:
#    print(db)

#mycursor.execute("""
#CREATE TABLE IF NOT EXISTS original (
#  video_id VARCHAR(255) PRIMARY KEY,
#  upload_date TEXT,
#  video_title TEXT,
#  video_tags TEXT,
#  video_duration TEXT,
#  video_view INTEGER,
#  video_like INTEGER,
#  video_comment INTEGER
#)""")

In [91]:
checkFormula = 'SELECT video_id FROM original WHERE video_id=%s'
updateForumla = 'UPDATE original SET video_view = %s, video_like = %s, video_comment = %s WHERE video_id = %s;'
insertForumla = 'INSERT INTO original (video_id, upload_date, video_title, video_tags, video_duration, video_view, video_like, video_comment) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);'

for i, row in df.iterrows():
    mycursor.execute(checkFormula, (row['videoId'], ))
    if mycursor.fetchone() is not None: #check if it exists
        #update
        rowData = (
            row['viewCount'],
            row['likeCount'],
            row['commentCount'],
            row['videoId']
        )
        mycursor.execute(updateForumla, rowData)
    else:
        #insert
        rowData = (
            row['videoId'],
            row['publishedAt'],
            row['title'],
            ', '.join(row['tags']),
            row['duration'],
            row['viewCount'],
            row['likeCount'],
            row['commentCount']
        )
        mycursor.execute(insertForumla, rowData)

mydb.commit()
